# 🚌 Projet MDM - Mobilité Durable en Montagne ⛰️

*Auteur : Thibault Launois*

*Date : 26/10/2025*

**Description :**

Ce script charge les données de **List_iti_D4G_isere_output.csv** (voir [ticket #3](https://github.com/data-for-good-grenoble/mobilite_durable/issues/3#issuecomment-3427357196)) et va créer un nouveau fichier avec l'altitude afin de réduire le nombre d'appel à l'API.

Pour faire les appels je fais tourner sur ma machine un serveur venant de [Open topo Data](https://www.opentopodata.org) avec les données de [Eudem 25m](https://www.opentopodata.org/datasets/eudem/).

# Module importation

In [1]:
import time

import numpy as np
import pandas as pd
import requests

# Load Data

In [2]:
df = pd.read_csv("../data/C2C/Liste_iti_D4G_isere_output.csv")

In [3]:
df.head(2)

,Titre itinéraire,Id itinéraire,url,activité,durée,dénivelé (m),longueur (m),Région/massif,Id wp principal,"(id WP - titre - [X,Y] - accessibilité (si renseigné) )",X,Y,lon,lat
0,Les Bans:Spigolo Sud du Contrefort Médian. Voi...,1780070,https://www.camptocamp.org/routes/1780070,mountain_climbing,['1'],NaN,0.0,(14328 - Isère)(14403 - Écrins),39056.0,"(39056 - Les Bans - [705350.3499286951, 55976...",705350.349929,5.597690e+06,6.336270,44.848420
1,Les Bans:Spigolo Sud du Contrefort Médian. Voi...,1780070,https://www.camptocamp.org/routes/1780070,mountain_climbing,['1'],NaN,0.0,(14328 - Isère)(14403 - Écrins),39056.0,(127458 - Entre Les Aygues - [712692.092985492...,712692.092985,5.594929e+06,6.402222,44.830833


# Get Elevations

In [4]:
def get_elevations(
    df, url, column_latitude="lat", column_longitude="lon", nbr_request_per_call=100, sleep=1
):
    """
    Will retrieve the elevations and add to the dataframe for each lines.

    Public API limits to 100 request locations at one time, and 1 second of delay between two calls and limits to 1000 calls per month
    """
    df["altitude"] = np.nan
    nbr_loop = len(df) // nbr_request_per_call
    for i in range(nbr_request_per_call, len(df), nbr_request_per_call):
        print(f"Loop: {i // nbr_request_per_call:2d}/{nbr_loop}", end="\r")
        df_slice = df[i - nbr_request_per_call : i]
        request = do_request(df_slice, url, column_latitude, column_longitude)
        unpack_request(df_slice, request, column_latitude, column_longitude)
        if sleep != 0:
            time.sleep(sleep)

    # last lines
    if i < len(df):
        df_slice = df[i:]
        request = do_request(df_slice, url, column_latitude, column_longitude)
        unpack_request(df_slice, request, column_latitude, column_longitude)


def do_request(df, url, column_latitude, column_longitude):
    """
    Concatenate the lat and lon of the lines in the dataframe and do a request to the server to get the altitudes
    Input:
        df (dataframe): dataframe with a latitude and longitude column
        url (str): url for the request
        col_lat_name (str): name of the column for the latitudes
        col_lon_name (str): name of the column for the longitudes

    Output:
        request output
    """
    request_str = ""

    # add all the latitude and longitude together
    for lat, lon in zip(df[column_latitude], df[column_longitude]):
        request_str = request_str + f"{lat},{lon}|"

    result = requests.get(url + request_str)

    return result.json()["results"]


def unpack_request(df, request, column_latitude, column_longitude):
    """
    Will add the elevations to the dataframe. A security check is done on the lat and lon to make sure it's the same.
    """
    for dic_request, row in zip(request, df.iterrows()):
        elevation = dic_request["elevation"]
        lat = dic_request["location"]["lat"]
        lon = dic_request["location"]["lng"]
        if row[1][column_latitude] == lat and row[1][column_longitude] == lon:
            df.loc[row[0], "altitude"] = elevation
        else:
            raise ValueError(
                "The server didn't gave the elevations in the same order as requested."
            )

In [5]:
url = r"http://localhost:5000/v1/eudem25m?locations="
get_elevations(df, url, nbr_request_per_call=100, sleep=0)

# Save data with altitude inside

In [7]:
df.to_csv("../data/C2C/Liste_iti_D4G_isere_output_altitude.csv")